# Parkway Project Use Case 1: Write Off Cases Prediction

## DATA PREPARATION

### Load Data

#### Import packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 500)
import warnings
warnings.filterwarnings(action='once')
import pickle
from imblearn.over_sampling import SMOTE

In [2]:
import os
os.chdir('../')
print(os.getcwd())

/home/jupyter/temp25Oct-Mok-cfc9083c3125ca7ce3bf62e5/PRS-PM-ISY5002-GROUP5/SystemCode


/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
%load_ext autoreload
%autoreload 2
from datapipeline import Datapipeline
dpl = Datapipeline()

#### Read data from file

In [4]:
file_dir = './Data Exploration/data/uc1/'
dict_X_train_file_paths = {
#    'GHL' : [file_dir + 'GHL_new_train_X_uc1.pkl'],
#    'MEH' : [file_dir + 'MEH_new_train_X_uc1.pkl'],
#    'PEH' : [file_dir + 'PEH_new_train_X_uc1.pkl'],
    'PNH' : [file_dir + 'PNH_new_train_X_uc1.pkl']
}
dict_y_train_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_train.pkl',
    'MEH' : file_dir + 'MEH_data_y_train.pkl',
    'PEH' : file_dir + 'PEH_data_y_train.pkl',
    'PNH' : file_dir + 'PNH_data_y_train.pkl'
}
dict_X_test_file_paths = {
    'GHL' : [file_dir + 'GHL_new_test_X_uc1.pkl'],
    'MEH' : [file_dir + 'MEH_new_test_X_uc1.pkl'],
    'PEH' : [file_dir + 'PEH_new_test_X_uc1.pkl'],
    'PNH' : [file_dir + 'PNH_new_test_X_uc1.pkl']
}
dict_y_test_file_paths = {
    'GHL' : file_dir + 'GHL_data_y_test.pkl',
    'MEH' : file_dir + 'MEH_data_y_test.pkl',
    'PEH' : file_dir + 'PEH_data_y_test.pkl',
    'PNH' : file_dir + 'PNH_data_y_test.pkl'
}

In [5]:
dict_df_X_train = {}
dict_df_y_train = {}
dict_df_X_test = {}
dict_df_y_test = {}

for hosp in dict_X_train_file_paths:
    dict_df_X_train[hosp] = pd.concat([pd.read_pickle(file_path)
                                       for file_path in dict_X_train_file_paths[hosp]])
    print(f'X_train {hosp} {dict_df_X_train[hosp].shape}')
    
    dict_df_y_train[hosp] = pd.read_pickle(dict_y_train_file_paths[hosp])
    print(f'y_train {hosp} {dict_df_y_train[hosp].shape}')

    dict_df_X_test[hosp] = pd.concat([pd.read_pickle(file_path)
                                      for file_path in dict_X_test_file_paths[hosp]])
    print(f'X_test {hosp} {dict_df_X_test[hosp].shape}')

    dict_df_y_test[hosp] = pd.read_pickle(dict_y_test_file_paths[hosp])
    print(f'y_test {hosp} {dict_df_y_test[hosp].shape}')

X_train PNH (79336, 3223)
y_train PNH (79336,)
X_test PNH (26445, 3223)
y_test PNH (26445,)


In [6]:
def convert_y(df):
    df1 = dpl.bin_column(df.to_frame(), col='WRITE_OFF', bin_thresh = [500])
    df1 = pd.get_dummies(df1['bin_WRITE_OFF'])
    df1 = df1.drop(0, axis=1)
    df1.columns = ['WRITE_OFF_LABEL']
    return df1

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
for hosp in dict_df_y_train:
    dict_df_y_train[hosp] = convert_y(dict_df_y_train[hosp])
    dict_df_y_test[hosp] = convert_y(dict_df_y_test[hosp])

## DECISION TREE

### Decision Tree WITHOUT SMOTE

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.tree import export_graphviz
from subprocess import call

In [9]:
criterion = 'entropy'
rand_seed = 0

In [10]:
for hosp in dict_df_X_train:
    model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"Accuracy on {hosp} training set without SMOTE decision tree: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"Accuracy on {hosp} test set without SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_dt_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'decision tree\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'decision tree\n', classification_report(dict_df_y_test[hosp], y_pred)) 

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PNH training set without SMOTE decision tree: 1.0
Accuracy on PNH test set without SMOTE decision tree: 0.9984117980714691


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_no_smote_dt_model.pkl'>
  if __name__ == '__main__':


PNH decision tree
 [[26382    16]
 [   26    21]]
PNH decision tree
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.57      0.45      0.50        47

    accuracy                           1.00     26445
   macro avg       0.78      0.72      0.75     26445
weighted avg       1.00      1.00      1.00     26445



### Decision Tree WITH SMOTE

#### Decision Tree WITH SMOTE SS = 0.1 ~ 1.0

In [11]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = DecisionTreeClassifier(criterion=criterion,random_state=rand_seed)
        model.fit(X_train_ss, y_train_ss)
        print(model)
        print(f"Accuracy on {hosp} training set with {sample} SMOTE decision tree: {model.score(X_train_ss, y_train_ss)}")
        print(f"Accuracy on {hosp} test set with {sample} SMOTE decision tree: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_dt_model.pkl'
        pickle.dump(model, open(filename, 'wb'))
        
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'decision tree \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'decision tree \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PNH training set with 0.1 SMOTE decision tree: 1.0
Accuracy on PNH test set with 0.1 SMOTE decision tree: 0.9974664397806768


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_0.1_smote_dt_model.pkl'>
  del sys.path[0]


PNH decision tree 
 [[26362    36]
 [   31    16]]
PNH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.31      0.34      0.32        47

    accuracy                           1.00     26445
   macro avg       0.65      0.67      0.66     26445
weighted avg       1.00      1.00      1.00     26445

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PNH training set with 0.2 SMOTE decision tree: 1.0
Accuracy on PNH test set with 0.2 SMOTE decision tree: 0.997693325770467


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_0.2_smote_dt_model.pkl'>
  del sys.path[0]


...

PNH decision tree 
 [[26365    33]
 [   27    20]]
PNH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.38      0.43      0.40        47

    accuracy                           1.00     26445
   macro avg       0.69      0.71      0.70     26445
weighted avg       1.00      1.00      1.00     26445

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PNH training set with 0.9 SMOTE decision tree: 1.0
Accuracy on PNH test set with 0.9 SMOTE decision tree: 0.997579882775572


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_0.9_smote_dt_model.pkl'>
  del sys.path[0]


PNH decision tree 
 [[26364    34]
 [   30    17]]
PNH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.33      0.36      0.35        47

    accuracy                           1.00     26445
   macro avg       0.67      0.68      0.67     26445
weighted avg       1.00      1.00      1.00     26445

DecisionTreeClassifier(criterion='entropy', random_state=0)
Accuracy on PNH training set with 1.0 SMOTE decision tree: 1.0
Accuracy on PNH test set with 1.0 SMOTE decision tree: 0.9974286254490452


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_1.0_smote_dt_model.pkl'>
  del sys.path[0]


PNH decision tree 
 [[26357    41]
 [   27    20]]
PNH decision tree 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.33      0.43      0.37        47

    accuracy                           1.00     26445
   macro avg       0.66      0.71      0.68     26445
weighted avg       1.00      1.00      1.00     26445



## LOGISTIC REGRESSION

### Logistic Regression Without SMOTE

In [12]:
from sklearn.linear_model import LogisticRegression

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
reg_strength = 0.01

In [14]:
for hosp in dict_df_X_train:
    model = LogisticRegression(C=reg_strength).fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    print(f"{hosp} Training set without SMOTE log reg score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")
    
    filename = file_dir + f'{hosp}_uc1_no_smote_logreg_model.pkl'
    pickle.dump(model, open(filename, 'wb'))
    
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.01)
PNH Training set without SMOTE log reg score: 0.9983109811434909
PNH Test set without SMOTE log reg score: 0.9981849120816789


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_no_smote_logreg_model.pkl'>
  


PNH log reg 
 [[26397     1]
 [   47     0]]
PNH log reg 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.00      0.00      0.00        47

    accuracy                           1.00     26445
   macro avg       0.50      0.50      0.50     26445
weighted avg       1.00      1.00      1.00     26445



### Logistic Regression With SMOTE

#### Logistic Regression With SMOTE ss=0.1 ~ 1.0

In [15]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        model = LogisticRegression(C=reg_strength).fit(X_train_ss, y_train_ss)
        print(model)
        print(f"{hosp} Training set with {sample} SMOTE log reg score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE log reg score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        filename = file_dir + f'{hosp}_uc1_{sample}_smote_logreg_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'log reg \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'log reg \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    htt

LogisticRegression(C=0.01)
PNH Training set with 0.1 SMOTE log reg score: 0.9679526176237646
PNH Test set with 0.1 SMOTE log reg score: 0.9951975798827756


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_0.1_smote_logreg_model.pkl'>
  if sys.path[0] == '':


PNH log reg 
 [[26288   110]
 [   17    30]]
PNH log reg 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.21      0.64      0.32        47

    accuracy                           1.00     26445
   macro avg       0.61      0.82      0.66     26445
weighted avg       1.00      1.00      1.00     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


...

PNH log reg 
 [[25938   460]
 [   17    30]]
PNH log reg 
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     26398
           1       0.06      0.64      0.11        47

    accuracy                           0.98     26445
   macro avg       0.53      0.81      0.55     26445
weighted avg       1.00      0.98      0.99     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=0.01)
PNH Training set with 1.0 SMOTE log reg score: 0.989892804383783
PNH Test set with 1.0 SMOTE log reg score: 0.9809415768576291


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_1.0_smote_logreg_model.pkl'>
  if sys.path[0] == '':


PNH log reg 
 [[25911   487]
 [   17    30]]
PNH log reg 
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     26398
           1       0.06      0.64      0.11        47

    accuracy                           0.98     26445
   macro avg       0.53      0.81      0.55     26445
weighted avg       1.00      0.98      0.99     26445



## NAIVE BAYES

### Naive Bayes Without SMOTE

In [16]:
from sklearn.naive_bayes import GaussianNB

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
for hosp in dict_df_X_train:
    # Initiating the Gaussian Classifier
    model = GaussianNB()

    # Training your model 
    model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
    print(model)
    filename = file_dir + f'{hosp}_uc1_no_smote_nb_model.pkl'
    pickle.dump(model, open(filename, 'wb'))

    # Score
    print(f"{hosp} Training set without SMOTE Naive Bayes score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    # Confusion Matrix
    y_pred = model.predict(dict_df_X_test[hosp])
    print(hosp, 'Naive Bayes \n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
    print(hosp, 'Naive Bayes \n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_no_smote_nb_model.pkl'>
  if __name__ == '__main__':


PNH Training set without SMOTE Naive Bayes score: 0.9658162750831905
PNH Test set without SMOTE Naive Bayes score: 0.9656267725467952
PNH Naive Bayes 
 [[25526   872]
 [   37    10]]
PNH Naive Bayes 
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     26398
           1       0.01      0.21      0.02        47

    accuracy                           0.97     26445
   macro avg       0.50      0.59      0.50     26445
weighted avg       1.00      0.97      0.98     26445



### Naive Bayes With SMOTE

#### Naive Bayes Without SMOTE ss=0.1 ~ 1.0

In [18]:
for hosp in dict_df_X_train:
    for sample in np.arange(0.1, 1.1, 0.1):
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Initiating the Gaussian Classifier
        model = GaussianNB()
        
        # Training your model 
        model.fit(X_train_ss, y_train_ss)
        print(model)
        filename = file_dir + f'{hosp}_uc1_{sample}_smote_nb_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        # Score
        print(f"{hosp} Training set with {sample} SMOTE Naive Bayes score: {model.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set with {sample} SMOTE Naive Bayes score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        # Confusion Matrix
        y_pred = model.predict(dict_df_X_test[hosp])
        print(hosp, 'Naive Bayes\n', confusion_matrix(dict_df_y_test[hosp], y_pred))  
        print(hosp, 'Naive Bayes\n', classification_report(dict_df_y_test[hosp], y_pred)) 

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_0.1_smote_nb_model.pkl'>
  del sys.path[0]


PNH Training set with 0.1 SMOTE Naive Bayes score: 0.9688708807291009
PNH Test set with 0.1 SMOTE Naive Bayes score: 0.9656267725467952
PNH Naive Bayes
 [[25526   872]
 [   37    10]]
PNH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     26398
           1       0.01      0.21      0.02        47

    accuracy                           0.97     26445
   macro avg       0.50      0.59      0.50     26445
weighted avg       1.00      0.97      0.98     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

PNH Training set with 0.9 SMOTE Naive Bayes score: 0.9819777912161668
PNH Test set with 0.9 SMOTE Naive Bayes score: 0.9656267725467952
PNH Naive Bayes
 [[25526   872]
 [   37    10]]
PNH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     26398
           1       0.01      0.21      0.02        47

    accuracy                           0.97     26445
   macro avg       0.50      0.59      0.50     26445
weighted avg       1.00      0.97      0.98     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


GaussianNB()


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_1.0_smote_nb_model.pkl'>
  del sys.path[0]


PNH Training set with 1.0 SMOTE Naive Bayes score: 0.9828790040529791
PNH Test set with 1.0 SMOTE Naive Bayes score: 0.9656267725467952
PNH Naive Bayes
 [[25526   872]
 [   37    10]]
PNH Naive Bayes
               precision    recall  f1-score   support

           0       1.00      0.97      0.98     26398
           1       0.01      0.21      0.02        47

    accuracy                           0.97     26445
   macro avg       0.50      0.59      0.50     26445
weighted avg       1.00      0.97      0.98     26445



## NEURAL NET MLP

### Neural Net MLP Without SMOTE, 2 Layers

In [19]:
from sklearn.neural_network import MLPClassifier  
from sklearn import metrics

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
list_hid_layers = [(128,128), (16,16,16), (32,32,32), (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

#### Neural Net MLPClassifier without SMOTE

In [21]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
        model.fit(dict_df_X_train[hosp], dict_df_y_train[hosp])
        print(model)
        filename = file_dir + f'{hosp}_uc1_no_smote_mlp_{hid_layers}_model.pkl'
        pickle.dump(model, open(filename, 'wb'))

        predictions = model.predict(dict_df_X_test[hosp]) 

        # Score
        print(f"{hosp} Training set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
        print(f"{hosp} Test set without SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy without SMOTE mlp {hid_layers}: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
        print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
        print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.02369188
Iteration 2, loss = 0.00664379
Iteration 3, loss = 0.00498875
Iteration 4, loss = 0.00366666
Iteration 5, loss = 0.00275576
Iteration 6, loss = 0.00217786
Iteration 7, loss = 0.00232103
Iteration 8, loss = 0.00137605
Iteration 9, loss = 0.00117535
Iteration 10, loss = 0.00080757
Iteration 11, loss = 0.00058167
Iteration 12, loss = 0.00084741
Iteration 13, loss = 0.00056601
Iteration 14, loss = 0.00046697
Iteration 15, loss = 0.00081881
Iteration 16, loss = 0.00040922
Iteration 17, loss = 0.00033487
Iteration 18, loss = 0.00039387
Iteration 19, loss = 0.00036052
Iteration 20, loss = 0.00036214
Iteration 21, loss = 0.00035305
Iteration 22, loss = 0.00112150
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(128, 128), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_no_smote_mlp_(128, 128)_model.pkl'>
  import sys


PNH Training set without SMOTE mlp (128, 128) score: 0.9997857214883533
PNH Test set without SMOTE mlp (128, 128) score: 0.9984117980714691
PNH Accuracy without SMOTE mlp (128, 128):  0.9984117980714691
PNH mlp (128, 128)
 [[26391     7]
 [   35    12]]
PNH mlp (128, 128)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.63      0.26      0.36        47

    accuracy                           1.00     26445
   macro avg       0.82      0.63      0.68     26445
weighted avg       1.00      1.00      1.00     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

PNH Training set without SMOTE mlp (16, 16, 16, 16) score: 0.9998739538166784
PNH Test set without SMOTE mlp (16, 16, 16, 16) score: 0.9984117980714691
PNH Accuracy without SMOTE mlp (16, 16, 16, 16):  0.9984117980714691
PNH mlp (16, 16, 16, 16)
 [[26388    10]
 [   32    15]]
PNH mlp (16, 16, 16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.60      0.32      0.42        47

    accuracy                           1.00     26445
   macro avg       0.80      0.66      0.71     26445
weighted avg       1.00      1.00      1.00     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.04257434
Iteration 2, loss = 0.00701941
Iteration 3, loss = 0.00551708
Iteration 4, loss = 0.00433360
Iteration 5, loss = 0.00315531
Iteration 6, loss = 0.00211594
Iteration 7, loss = 0.00187811
Iteration 8, loss = 0.00136336
Iteration 9, loss = 0.00099859
Iteration 10, loss = 0.00104321
Iteration 11, loss = 0.00069684
Iteration 12, loss = 0.00102306
Iteration 13, loss = 0.00063844
Iteration 14, loss = 0.00038129
Iteration 15, loss = 0.00039224
Iteration 16, loss = 0.00061034
Iteration 17, loss = 0.00040583
Iteration 18, loss = 0.00041016
Iteration 19, loss = 0.00073994
Iteration 20, loss = 0.00019964
Iteration 21, loss = 0.00029067
Iteration 22, loss = 0.00022844
Iteration 23, loss = 0.00051644
Iteration 24, loss = 0.00118663
Iteration 25, loss = 0.00022511
Iteration 26, loss = 0.00020899
Iteration 27, loss = 0.00018479
Iteration 28, loss = 0.00017268
Iteration 29, loss = 0.00014489
Iteration 30, loss = 0.00130290
Iteration 31, loss = 0.00054703
Training loss did

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_no_smote_mlp_(32, 32, 32, 32)_model.pkl'>
  import sys


PNH Training set without SMOTE mlp (32, 32, 32, 32) score: 0.9999495815266713
PNH Test set without SMOTE mlp (32, 32, 32, 32) score: 0.9984874267347325
PNH Accuracy without SMOTE mlp (32, 32, 32, 32):  0.9984874267347325
PNH mlp (32, 32, 32, 32)
 [[26385    13]
 [   27    20]]
PNH mlp (32, 32, 32, 32)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.61      0.43      0.50        47

    accuracy                           1.00     26445
   macro avg       0.80      0.71      0.75     26445
weighted avg       1.00      1.00      1.00     26445



### Neural Net With SMOTE

In [22]:
list_hid_layers = [(16,16), (32,32), (64,64), (128,128), (256,256),
                   (16,16,16), (32,32,32), (64,64,64),
                   (16,16,16,16), (32,32,32,32)]
max_iter = 1000
list_smote_sampling = [0.1, 0.2, 0.3, 0.5, 1.0]

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#### Neural Net Layers With SMOTE

In [ ]:
for hosp in dict_df_X_train:
    for hid_layers in list_hid_layers:
        for sample in list_smote_sampling:
            sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
            X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
            model = MLPClassifier(hidden_layer_sizes=hid_layers, max_iter=max_iter,verbose=1)  
            model.fit(X_train_ss, y_train_ss)  
            print(model)
            filename = file_dir + f'{hosp}_uc1_{sample}_smote_mlp_{hid_layers}_model.pkl'
            pickle.dump(model, open(filename, 'wb'))
            
            predictions = model.predict(dict_df_X_test[hosp]) 

            # Score
            print(f"{hosp} Training set {sample} SMOTE mlp {hid_layers} score: {model.score(X_train_ss, y_train_ss)}")
            print(f"{hosp} Test set {sample} SMOTE mlp {hid_layers} score: {model.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

            print(f"{hosp} Accuracy: ", metrics.accuracy_score(dict_df_y_test[hosp], predictions))
            print(hosp, f'mlp {hid_layers}\n', confusion_matrix(dict_df_y_test[hosp],predictions))  
            print(hosp, f'mlp {hid_layers}\n', classification_report(dict_df_y_test[hosp],predictions))  

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.19823630
Iteration 2, loss = 0.01140996
Iteration 3, loss = 0.00496564
Iteration 4, loss = 0.00302236
Iteration 5, loss = 0.00214177
Iteration 6, loss = 0.00164094
Iteration 7, loss = 0.00208556
Iteration 8, loss = 0.00109772
Iteration 9, loss = 0.00097542
Iteration 10, loss = 0.00136329
Iteration 11, loss = 0.00080532
Iteration 12, loss = 0.00061287
Iteration 13, loss = 0.00062073
Iteration 14, loss = 0.00048507
Iteration 15, loss = 0.00089327
Iteration 16, loss = 0.00047039
Iteration 17, loss = 0.00058500
Iteration 18, loss = 0.00058301
Iteration 19, loss = 0.00050527
Iteration 20, loss = 0.00061565
Iteration 21, loss = 0.00036779
Iteration 22, loss = 0.00055779
Iteration 23, loss = 0.00035657
Iteration 24, loss = 0.00029523
Iteration 25, loss = 0.00045916
Iteration 26, loss = 0.00097147
Iteration 27, loss = 0.00030684
Iteration 28, loss = 0.00023922
Iteration 29, loss = 0.00051327
Iteration 30, loss = 0.00054974
Iteration 31, loss = 0.00027049
Iteration 32, los

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_0.1_smote_mlp_(16, 16)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


PNH Training set 0.1 SMOTE mlp (16, 16) score: 0.9999655651335498
PNH Test set 0.1 SMOTE mlp (16, 16) score: 0.997579882775572
PNH Accuracy:  0.997579882775572
PNH mlp (16, 16)
 [[26366    32]
 [   32    15]]
PNH mlp (16, 16)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.32      0.32      0.32        47

    accuracy                           1.00     26445
   macro avg       0.66      0.66      0.66     26445
weighted avg       1.00      1.00      1.00     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

Iteration 1, loss = 0.05430336
Iteration 2, loss = 0.00280441
Iteration 3, loss = 0.00216407
Iteration 4, loss = 0.00067768
Iteration 5, loss = 0.00052838
Iteration 6, loss = 0.00059990
Iteration 7, loss = 0.00094968
Iteration 8, loss = 0.00044136
Iteration 9, loss = 0.00035122
Iteration 10, loss = 0.00036307
Iteration 11, loss = 0.00110331
Iteration 12, loss = 0.00039536
Iteration 13, loss = 0.00032684
Iteration 14, loss = 0.00032857
Iteration 15, loss = 0.00027237
Iteration 16, loss = 0.00029442
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
MLPClassifier(hidden_layer_sizes=(64, 64), max_iter=1000, verbose=1)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedWriter name='./Data Exploration/data/uc1/PNH_uc1_0.3_smote_mlp_(64, 64)_model.pkl'>
  # This is added back by InteractiveShellApp.init_path()


PNH Training set 0.3 SMOTE mlp (64, 64) score: 0.9999708627538583
PNH Test set 0.3 SMOTE mlp (64, 64) score: 0.9981470977500473
PNH Accuracy:  0.9981470977500473
PNH mlp (64, 64)
 [[26385    13]
 [   36    11]]
PNH mlp (64, 64)
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     26398
           1       0.46      0.23      0.31        47

    accuracy                           1.00     26445
   macro avg       0.73      0.62      0.65     26445
weighted avg       1.00      1.00      1.00     26445



/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Iteration 1, loss = 0.05027976
Iteration 2, loss = 0.00217943
Iteration 3, loss = 0.00099615
Iteration 4, loss = 0.00082900
Iteration 5, loss = 0.00072407
Iteration 6, loss = 0.00063415
Iteration 7, loss = 0.00050784
Iteration 8, loss = 0.00041847
Iteration 9, loss = 0.00047651
Iteration 10, loss = 0.00081472
Iteration 11, loss = 0.00035107
Iteration 12, loss = 0.00058820
Iteration 13, loss = 0.00035129
Iteration 14, loss = 0.00040140
Iteration 15, loss = 0.00029341
Iteration 17, loss = 0.00111806


## KNN

In [ ]:
# Be careful when running this, about 2 full days are needed to run this!!!
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
for hosp in dict_df_X_train:
    test_scores = []
    train_scores = []

    for i in range(1,15):

        knn = KNeighborsClassifier(i)
        knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])

        train_scores.append(knn.score(dict_df_X_train[hosp],dict_df_y_train[hosp]))
        test_scores.append(knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp]))

    ## score that comes from testing on the same datapoints that were used for training
    max_train_score = max(train_scores)
    train_scores_ind = [i for i, v in enumerate(train_scores) if v == max_train_score]
    print(f'{hosp} Max train score {max_train_score*100} % and k = {list(map(lambda x: x+1, train_scores_ind))}')

    ## score that comes from testing on the datapoints that were split in the beginning to be used for testing solely
    max_test_score = max(test_scores)
    test_scores_ind = [i for i, v in enumerate(test_scores) if v == max_test_score]
    print(f'{hosp} Max test score {max_test_score*100} % and k = {list(map(lambda x: x+1, test_scores_ind))}')

    plt.figure(figsize=(12,5))
    plt.plot(range(1,15),train_scores,marker='*',label='Train Score')
    plt.plot(range(1,15),test_scores,marker='o',label='Test Score')
    plt.title(f'{hosp} KNN')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# From the above, the best KNN is at k=8
#Setup a knn classifier with k neighbors
best_knn = KNeighborsClassifier(8)

### KNN without SMOTE

In [ ]:
n_neighbours = 8
for hosp in dict_df_X_train:
    # Fit the KNN model with training data and score the test data
    best_knn = KNeighborsClassifier(n_neighbours)
    
    best_knn.fit(dict_df_X_train[hosp],dict_df_y_train[hosp])
    best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
    
    # save the model to disk
    filename = f'{hosp}_best_knn_without_smote.pkl'
    pickle.dump(best_knn, open(filename, 'wb'))
    
    y_pred = best_knn.predict(dict_df_X_test[hosp])
    # Score for Best KNN, No SMOTE
    print(f"{hosp} Training set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_train[hosp], dict_df_y_train[hosp])}")
    print(f"{hosp} Test set without SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

    print(f"{hosp} Accuracy without SMOTE KNN({n_neighbours}): {metrics.accuracy_score(dict_df_y_test[hosp], y_pred)}")
    print(hosp, f'KNN({n_neighbours})\n', confusion_matrix(dict_df_y_test[hosp],y_pred))  
    print(hosp, f'KNN({n_neighbours})\n', classification_report(dict_df_y_test[hosp],y_pred))  

### KNN With SMOTE

#### KNN With SMOTE ss=0.1

In [ ]:
n_neighbours = 8
list_smote_sampling = [0.1, 0.5, 1.0]
for hosp in dict_df_X_train:
    for sample in list_smote_sampling:
        sm_ss = SMOTE(random_state=55,sampling_strategy=sample)
        X_train_ss, y_train_ss = sm_ss.fit_sample(dict_df_X_train[hosp], dict_df_y_train[hosp])
        
        # Fit the KNN model with training data and score the test data
        best_knn = KNeighborsClassifier(n_neighbours)    
        best_knn.fit(X_train_ss,y_train_ss)
        #best_knn.score(dict_df_X_test[hosp],dict_df_y_test[hosp])
        
        # save the model to disk
        filename = f'{hosp}_best_knn_{sample}_smote.pkl'
        pickle.dump(best_knn, open(filename, 'wb'))
        
        y_pred = best_knn.predict(dict_df_X_test[hosp])
        # Score for Best KNN, with SMOTE
        print(f"{hosp} Training set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(X_train_ss, y_train_ss)}")
        print(f"{hosp} Test set {sample} SMOTE KNN({n_neighbours}) score: {best_knn.score(dict_df_X_test[hosp], dict_df_y_test[hosp])}")

        print(f"{hosp} Accuracy {sample} SMOTE KNN({n_neighbours}): ", metrics.accuracy_score(dict_df_y_test[hosp], y_pred))
        print(confusion_matrix(dict_df_y_test[hosp],y_pred))  
        print(classification_report(dict_df_y_test[hosp],y_pred)) 